In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="jOcb5Y1QDeocxMBEA0Bi")
project = rf.workspace("dstu-dfliv").project("palm-lines-recognition")
version = project.version(10)
dataset = version.download("yolov8")

In [ ]:
!pip install -U ultralytics

In [ ]:
import os
import cv2
import shutil
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

# 1. 커스텀 MobileNetV3-Pose 모델 설정 (YAML 생성)
# MobileNetV3-Small을 백본으로 사용하여 속도와 효율을 극대화합니다.
mobilenet_yaml = """
# MobileNetV3-Small Pose Config
nc: 5  # 데이터셋의 클래스 수 (palm-lines-recognition 기준)
kpt_shape: [5, 2]  # 손금 주요 지점 수 (데이터셋에 맞게 자동 조정됨)

backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [16, 3, 2]]  # 0-P1/2
  - [-1, 1, Conv, [16, 3, 1]]  # 1
  - [-1, 1, Conv, [24, 3, 2]]  # 2-P2/4
  - [-1, 1, Conv, [24, 3, 1]]  # 3
  - [-1, 1, Conv, [40, 3, 2]]  # 4-P3/8
  - [-1, 3, Conv, [40, 3, 1]]  # 5
  - [-1, 1, Conv, [80, 3, 2]]  # 6-P4/16
  - [-1, 3, Conv, [80, 3, 1]]  # 7
  - [-1, 1, Conv, [112, 3, 1]] # 8
  - [-1, 1, Conv, [112, 3, 1]] # 9
  - [-1, 1, Conv, [160, 3, 2]] # 10-P5/32
  - [-1, 1, SPPF, [1024, 5]]   # 11

head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 9], 1, Concat, [1]]  # cat backbone P4
  - [-1, 3, C2f, [112]]        # 14

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 5], 1, Concat, [1]]  # cat backbone P3
  - [-1, 3, C2f, [40]]         # 17 (P3/8-small)

  - [-1, 1, Conv, [40, 3, 2]]
  - [[-1, 14], 1, Concat, [1]] # cat head P4
  - [-1, 3, C2f, [112]]        # 20 (P4/16-medium)

  - [-1, 1, Conv, [112, 3, 2]]
  - [[-1, 11], 1, Concat, [1]] # cat head P5
  - [-1, 3, C2f, [256]]        # 23 (P5/32-large)

  - [[17, 20, 23], 1, Pose, [nc, kpt_shape]] # Pose head
"""

with open('mobilenet_pose.yaml', 'w') as f:
    f.write(mobilenet_yaml)

# 2. 명암 대비 전처리 (CLAHE 1.2 - 낮게 설정)
def apply_low_clahe(image_path):
    img = cv2.imread(image_path)
    if img is None: return None
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    # clipLimit를 1.2로 낮춰서 노이즈를 억제하고 선만 뚜렷하게 함
    clahe = cv2.createCLAHE(clipLimit=1.2, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    return cv2.cvtColor(cv2.merge((cl, a, b)), cv2.COLOR_LAB2BGR)

# 3. 모델 학습 (100에폭, Mosaic/Mixup Off)
model = YOLO('mobilenet_pose.yaml') # 위에서 만든 MobileNet 구조 로드

model.train(
    data='Palm-lines-recognition-10/data.yaml',
    epochs=100,
    imgsz=640,
    mosaic=0.0,   # 모자이크 끄기
    mixup=0.0,    # 믹스업 끄기
    name='palm_mobilenet',
    save=True
)

# 4. 베스트 모델을 num_02.pt로 저장
best_path = os.path.join(model.trainer.save_dir, 'weights/best.pt')
shutil.copy(best_path, 'num_02.pt')
print("✅ MobileNet-Pose 모델 저장 완료: num_02.pt")

# 5. 시각화 함수 (같은 클래스 동일 색상 적용)
def visualize_test(image_path):
    processed_img = apply_low_clahe(image_path)
    trained_model = YOLO('num_02.pt')
    results = trained_model.predict(processed_img, conf=0.25)[0]

    names = results.names
    # 고정 색상 맵 (클래스별로 다른 색상 지정)
    cmap = plt.cm.get_cmap('hsv', len(names))

    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB))
    ax = plt.gca()

    if results.boxes is not None:
        for i, box in enumerate(results.boxes):
            cls_id = int(box.cls[0])
            color = cmap(cls_id)

            # 박스 및 키포인트(히트맵 기반 포인트) 표시
            b = box.xyxy[0].cpu().numpy()
            ax.add_patch(plt.Rectangle((b[0], b[1]), b[2]-b[0], b[3]-b[1],
                                     fill=False, edgecolor=color, linewidth=2))

            if results.keypoints is not None:
                pts = results.keypoints.xy[i].cpu().numpy()
                for pt in pts:
                    if pt[0] > 0:
                        plt.scatter(pt[0], pt[1], color=color, s=30, edgecolors='white')

            ax.text(b[0], b[1]-5, f"{names[cls_id]}", color='white', fontsize=10,
                    bbox=dict(facecolor=color, alpha=0.8, edgecolor='none'))

    plt.axis('off')
    plt.show()

# 실행 예시 (테스트용 이미지 경로 입력)
# visualize_test('test_image.jpg')

In [ ]:
#시각화
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from google.colab import files  # 코랩 전용 파일 업로드 모듈
import io

def colab_visualize_palm(model_path='num_02.pt'):
    # 1. 파일 업로드 인터페이스 띄우기
    print("시각화할 손 이미지를 업로드하세요.")
    uploaded = files.upload()

    if not uploaded:
        print("파일이 선택되지 않았습니다.")
        return

    # 2. 모델 로드 (학습된 num_02.pt)
    # 만약 파일이 없다면 num01.pt나 yolo11n-pose.pt 등으로 이름을 바꿔주세요.
    try:
        model = YOLO(model_path)
    except:
        print(f"❌ {model_path} 파일을 찾을 수 없습니다. 파일 이름을 확인해주세요.")
        return

    # 3. 업로드된 파일 처리
    for filename in uploaded.keys():
        # 이미지 읽기
        file_bytes = np.frombuffer(uploaded[filename], dtype=np.uint8)
        img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)

        if img is None: continue

        # 4. 전처리 (CLAHE 1.2 - 낮은 강도)
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=1.2, tileGridSize=(8, 8))
        cl = clahe.apply(l)
        processed_img = cv2.cvtColor(cv2.merge((cl, a, b)), cv2.COLOR_LAB2BGR)

        # 5. 모델 예측
        results = model.predict(processed_img, conf=0.25)[0]
        names = results.names

        # 6. 클래스별 고정 색상 설정 (동일 클래스 = 동일 색상)
        class_colors = [
            (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0),
            (255, 0, 255), (0, 255, 255), (255, 165, 0), (128, 0, 128)
        ]

        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB))
        ax = plt.gca()

        if results.boxes is not None:
            for i, box in enumerate(results.boxes):
                cls_id = int(box.cls[0])
                base_color = class_colors[cls_id % len(class_colors)]
                color_plt = tuple(c/255 for c in base_color)

                # 박스 시각화
                b = box.xyxy[0].cpu().numpy()
                ax.add_patch(plt.Rectangle((b[0], b[1]), b[2]-b[0], b[3]-b[1],
                                         fill=False, edgecolor=color_plt, linewidth=3))

                # 키포인트(손금 점) 시각화
                if results.keypoints is not None:
                    kpts = results.keypoints.xy[i].cpu().numpy()
                    for pt in kpts:
                        if pt[0] > 0:
                            plt.scatter(pt[0], pt[1], color=color_plt, s=40, edgecolors='white', zorder=5)

                # 라벨 배경색 고정
                ax.text(b[0], b[1]-10, f"{names[cls_id]}", color='white', fontweight='bold',
                        bbox=dict(facecolor=color_plt, edgecolor='none', alpha=0.8))

        plt.axis('off')
        plt.title(f"Result: {filename}", fontsize=12)
        plt.show()

# 실행
colab_visualize_palm('num_02.pt')

In [ ]:
# 1. 라이브러리 설치
!pip install ultralytics

import cv2
import numpy as np
from ultralytics import YOLO
from google.colab import files
import matplotlib.pyplot as plt

# 2. 모델 로드
model_path = '/content/num_02.pt'
model = YOLO(model_path)

# 3. 클래스별 고유 색상 매핑 (BGR 순서: (B, G, R))
# 모델의 클래스 순서에 맞춰 원하는 색상을 지정하세요.
# 예: 0번은 빨강, 1번은 노랑, 2번은 파랑...
CLASS_COLORS = [
    (0, 0, 255),    # 0번: 빨강
    (0, 255, 255),  # 1번: 노랑
    (255, 0, 0),    # 2번: 파랑
    (0, 255, 0),    # 3번: 초록
    (255, 0, 255),  # 4번: 보라
    (255, 255, 0),  # 5번: 하늘색
]

# 4. 사진 업로드
print("손금 사진을 업로드해주세요.")
uploaded = files.upload()

FATE_KEYWORD = 'fate' # 모델에 따라 '운명선' 등으로 변경 가능

for filename in uploaded.keys():
    results = model.predict(source=filename, conf=0.2, save=False)

    img = cv2.imread(filename)
    names = model.names
    fate_ids = [k for k, v in names.items() if FATE_KEYWORD in v.lower()]

    for r in results:
        if r.keypoints is None: continue

        kpts = r.keypoints.data.cpu().numpy()
        classes = r.boxes.cls.cpu().numpy().astype(int)

        fate_kpts_list = []
        fate_color = (255, 255, 255)

        # --- 일반 손금 및 데이터 수집 ---
        for i, cls_id in enumerate(classes):
            color = CLASS_COLORS[cls_id % len(CLASS_COLORS)]

            if cls_id in fate_ids:
                fate_kpts_list.append(kpts[i])
                fate_color = color
            else:
                # 점(Keypoint)과 선 그리기
                points = kpts[i]
                for j in range(len(points)):
                    x, y, conf = points[j]
                    if conf > 0.3: # 신뢰도 기준
                        # 1. 선 그리기 (다음 점과 연결)
                        if j < len(points) - 1:
                            nx, ny, nconf = points[j+1]
                            if nconf > 0.3:
                                cv2.line(img, (int(x), int(y)), (int(nx), int(ny)), color, 5)
                        # 2. 점 그리기 (원하는 스타일)
                        cv2.circle(img, (int(x), int(y)), 8, color, -1) # 채워진 원
                        cv2.circle(img, (int(x), int(y)), 10, (255, 255, 255), 2) # 테두리

        # --- 운명선 통합 (두 줄을 하나로) ---
        if len(fate_kpts_list) > 0:
            merged_fate = np.mean(fate_kpts_list, axis=0)
            for j in range(len(merged_fate)):
                x, y, conf = merged_fate[j]
                if conf > 0.1:
                    if j < len(merged_fate) - 1:
                        nx, ny, nconf = merged_fate[j+1]
                        if nconf > 0.1:
                            cv2.line(img, (int(x), int(y)), (int(nx), int(ny)), fate_color, 8) # 더 두껍게
                    cv2.circle(img, (int(x), int(y)), 10, fate_color, -1)
                    cv2.circle(img, (int(x), int(y)), 12, (255, 255, 255), 2)

    # 결과 시각화
    plt.figure(figsize=(15, 10))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()